# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

Q1: Any test for proportion differences/ratio, such as odds ratio test or fisher exact test. CLT most likely applies here. We can assume each resume is for different person, and hopefully different people reviewed each of them, thus we have independency. Cohort sizes are large enough.  
Q2: H0: There is no difference in proportion of calls between two race groups. or OR = 1.
H1: OR != 1

In [5]:
w = data[data.race=='w']
b = data[data.race=='b']

In [6]:
# Your solution to Q3 here
from scipy import stats
tbl = np.array([[np.sum(w.call), np.sum(1-w.call)], [np.sum(b.call), np.sum(1-b.call)]])
print(tbl)
stats.fisher_exact(tbl)

[[ 235. 2200.]
 [ 157. 2278.]]


(1.5498841922408801, 4.758747107918147e-05)

Crude test result indicate a significant different with OR = 1.5

In [7]:
# bootstrap statistics
def get_bs_or(data, size):
    ind = np.arange(data.shape[0])
    def get_single_bs_or():
        bs = data.iloc[np.random.choice(ind, size=len(ind), replace=True)]
        w = bs[bs.race=='w']
        b = bs[bs.race=='b']
        oddr = np.sum(w.call)/np.sum(1-w.call)/np.sum(b.call)*np.sum(1-b.call)
        return(oddr)
    return(np.array([get_single_bs_or() for i in range(size)]))
test_or = get_bs_or(data, 1000)
ci = np.percentile(test_or, [2.5, 97.5])
p = np.sum(test_or <= 1)/len(test_or)
print(ci)
print(p)
# this is not exact but good enough

[1.24497404 1.91449417]
0.0


<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

In [8]:
white_prop = np.sum(w.call)/w.shape[0]
black_prop = np.sum(b.call)/b.shape[0]
print(white_prop, black_prop)

0.09650924024640657 0.06447638603696099


9.7% of resumes with 'white' names got call-backs comparing to 6.4% among resumes with 'black' names. The difference is statistically different, indicating the existence of racial discrimination that black people are less likely to get job interviews.

But this doesn't mean if 'race' indicated by name is the most influencial factor of rate of getting job interview. There can be other factors strongly associated with rate of calls. For example, higher education level might be more positively associated with chance of interview, while a lower skill requirement on the job will also strongly associated with chance of interview. The easiest way is to run a multivariate analysis, such as logistic regression in this case to identify other factors.